\begin{equation*}
\begin{aligned}
& \underset{i_{el}(t)}{\text{minimize}}
& &  \int_{0}^{T_f} (p_{el}(t))-p_{ph}(t))^2 \,dt \\
& \text{subject to}
& & \dot m_{H_2} = f_{H_2} - d_{H_2}, \\
&&& f_{H_2} = N_{el}\frac{i_{el}}{F}, \\
&&& v_{el}=v_{el,0}+v_{etd}+v_{ohm}+v_{ion}, \\
&&& v_{el,0}=1.23-0.9\times10^{-3}\left(t_{el}-298\right)+2.3 \frac{R t_{el}}{4 F} \ln \left(P_{H_2}^2 P_{O_2}\right), \\
&&& V_{etd}=\frac{R t_{el}}{F} \sinh ^{-1}\left[\frac{1}{2} \frac{i_{el}}{i_{ao}}\right]+\frac{R t_{ez}}{F} \sinh ^{-1}\left[\frac{1}{2} \frac{i_{el}}{i_{co}}\right]+\left(\frac{\delta_B}{\sigma_B} i_{el}\right)+R_I i_{el}, \\
&&& v_{ez,ohm}+v_{ion}=\frac{\delta_B i_{el}}{A_{el} \sigma_B}, \\
&&& \sigma_B=(0.005139 \lambda-0.00326) e^{1268}\left(\frac{1}{303}-\frac{1}{t_{el}}\right), \\
&&& -1 \leq u \leq 1.
\end{aligned}
\end{equation*}

In [ ]:
# Imports
import casadi as ca

# Declare constants
R = 8.314 # Gas constant
F = 96485 # Faraday constant

# Declare electrolyzer parameters
A_el = 212.5        # Stack area
N_el = 1            # Number of cells
P_h2 = 6.9          # Hydrogen partial pressure
P_o2 = 1.3          # Oxygen partial pressure
I_ao = 1.0631e-6    # Anode current density 
I_co = 1e-3         # Cathode current density
delta_b = 178.5     # Membrane thickness
lambda_b = 21       # Membrana water content
C_el = 0.5          # Thermal capacitance 
tau_el = 0          # NÃO ACHEI NO ARTIGO !!!
P_el = 0            # NÃO ACHEI NO ARTIGO !!!
R_I = 0             # NÃO ACHEI NO ARTIGO !!!

# Declare variables
m_h2 = ca.SX.sym('m_h2') # State - Mass of hydrogen
t_el = ca.SX.sym('t_el') # State - Temperature of electrolyzer
t_ab = ca.SX.sym('t_ab') # Algebric - Ambient temperature
f_h2 = ca.SX.sym('f_h2') # Algebric - Produced hydrogen in the electrolyzer
d_h2 = ca.SX.sym('d_h2') # Algebric - Hydrogen demand
i_ph = ca.SX.sym('i_pv') # Algebric - Electrical current in photovoltaic panel
v_ph = ca.SX.sym('v_pv') # Algebric - Voltage of photovoltaic panel
v_el = ca.SX.sym('v_el') # Algebric - Voltage of electrolyzer
i_el = ca.SX.sym('i_el') # Control - Electrical current in electrolyzer

# Intermediate electrolyzer variables - Electric
v_el_0 = 1.23 - 0.0009*(t_el-298) + 2.3*R*t_el*ca.log(P_h2**2*P_o2)/(4*F) # Reversible potential of the electrolyzer
ro_b = (0.005139*lambda_b - 0.00326) * ca.exp(1268*(1/303 - 1/t_el)) # Membrane conductivity
v_etd = (R*t_el/F)*ca.sinh(.5*i_el/(I_ao))**-1 + (R*t_el/F)*ca.sinh(.5*i_el/(I_co))**-1 + i_el*delta_b/ro_b + R_I*i_el # Eletrode overpotential
v_el_hom_ion = delta_b*i_el/(A_el*ro_b) # Ohmic overvoltage and ionic overpotential

# Intermediate electrolyzer variables - Thermal
p_sat = ca.exp(13.669-(8096.23/t_el)) # Vapor partial pressure
v_hv = 1.4756 + 2.252e-4*(t_el-273) + 1.52e-8*(t_el-273)**2 # Higher-high-value voltage
v_tn = v_hv + (1.5/(2*F))*(p_sat/(P_el + p_sat))*(4.29e-4 + 40.76*(t_el-273) - 0.066*(t_el-273)**2) # Thermal voltage
q_gen = v_el*i_el*(1-v_tn) # Heat generated
q_loss = (t_el - t_ab)*C_el/tau_el # Heat loss to the environment
q_cool = 0 # Heat removed by the cooling system NÃO ACHEI NO ARTIGO !!!

# Diferential equations
m_h2_dot = f_h2 - d_h2
t_el_dot = (q_gen + q_loss + q_cool)/C_el
f_x = ca.vertcat(m_h2_dot, t_el_dot)

# Algebraic equations
f_h2 = N_el*i_el/F
v_el = v_el_0 + v_etd + v_el_hom_ion
f_z = ca.vertcat(f_h2, v_el)

# Lagrange cost function
f_q = (v_el*i_el - v_ph*i_ph)**2